In [1]:
import pandas as pd
import recordlinkage
import jellyfish
import json

# Caricamento dei dati
df = pd.read_csv("main_outputs/final_mediated_schema.csv", low_memory=False)

# Assicurati che il nome dell'azienda sia stringa
df["company_name"] = df["company_name"].astype(str)

# Creazione di un identificativo unico per ogni record
df["index"] = df.index

# Strategie di Blocking
indexer = recordlinkage.Index()

# Blocking 1: Basato sulle prime 3 lettere del nome dell'azienda
df["prefix_3"] = df["company_name"].str[:3]
indexer.block(left_on="prefix_3")

# Blocking 2: Basato su Soundex (usando jellyfish)
df["soundex"] = df["company_name"].apply(lambda x: jellyfish.soundex(x))
indexer.block(left_on="soundex")

# Creazione dei candidati per il matching
candidate_links_1 = indexer.index(df)
candidate_links_2 = indexer.index(df)

# Funzione per trasformare gli ID in valori leggibili
def convert_pairs_to_json(candidate_links, filename):
    pairs_list = []
    
    for id1, id2 in candidate_links:
        row1 = df.loc[id1]
        row2 = df.loc[id2]
        
        pairs_list.append({
            "company_1": row1["company_name"],
            "company_2": row2["company_name"],
            "source_1": row1["_source_table"] if "_source_table" in df.columns else None,
            "source_2": row2["_source_table"] if "_source_table" in df.columns else None
        })
    
    # Salva in JSON
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(pairs_list, f, indent=4, ensure_ascii=False)

# Salvare i risultati del blocking come JSON leggibile
convert_pairs_to_json(candidate_links_1, "blocking_output_1.json")
convert_pairs_to_json(candidate_links_2, "blocking_output_2.json")

print("Blocking completato e salvato in JSON!")

Blocking completato e salvato in JSON!


In [6]:
import pandas as pd
import recordlinkage
import json
from difflib import SequenceMatcher
from sklearn.metrics import precision_score, recall_score, f1_score

# Funzione per calcolare la similarità tra stringhe
def similar(a, b) -> float:
    return SequenceMatcher(None, a, b).ratio()

# Caricamento della ground truth
with open("data/ground_truth.json", "r", encoding="utf-8") as f:
    ground_truth = json.load(f)["ground_truth"]

# Convertiamo la ground truth in un set di tuple per confrontarle più facilmente
gt_pairs = set((list(gt.keys())[0], list(gt.keys())[1]) for gt in ground_truth)

# Caricamento delle coppie generate dal blocking
with open("blocking_output_1.json", "r", encoding="utf-8") as f:
    candidate_pairs = json.load(f)

# Caricamento del dataset principale
df = pd.read_csv("main_outputs/final_mediated_schema.csv", low_memory=False)

# Creiamo un DataFrame per le coppie candidate
pairs_df = pd.DataFrame(candidate_pairs)

# Creiamo un indice numerico per unire i dati più facilmente
df = df.reset_index()

# Creiamo un dizionario per mappare i nomi aziendali agli ID del DataFrame
company_to_index = df.set_index("company_name")["index"].to_dict()

# Convertiamo le coppie candidate in un MultiIndex di ID
pairs_idx = [
    (company_to_index.get(row["company_1"]), company_to_index.get(row["company_2"]))
    for _, row in pairs_df.iterrows()
    if row["company_1"] in company_to_index and row["company_2"] in company_to_index
]

# Creiamo un MultiIndex valido
pairs_index = pd.MultiIndex.from_tuples(pairs_idx, names=["id1", "id2"])

# Creazione del confronto tra le coppie di dati
compare = recordlinkage.Compare()
compare.string("company_name", "company_name", method="jarowinkler", threshold=0.85, label="name_similarity")

# Ora passiamo correttamente il MultiIndex come primo argomento
df_indexed = df.set_index("index")  # Assicuriamoci che il DataFrame sia indicizzato correttamente
features = compare.compute(pairs_index, df_indexed)  # Ora funziona

# Selezioniamo le coppie che superano la soglia di similarità
threshold = 0.95
matched_pairs = set(
    (df.loc[id1, "company_name"], df.loc[id2, "company_name"])
    for (id1, id2), row in features.iterrows()
    if row["name_similarity"] > threshold
)

# Creazione delle liste per le metriche
y_true = [1 if pair in gt_pairs else 0 for pair in matched_pairs]
y_pred = [1] * len(matched_pairs)  # Tutte le coppie trovate sono considerate come predizioni positive

# Calcolo delle metriche
precision = precision_score(y_true, y_pred) if len(set(y_true)) > 1 else 0
recall = recall_score(y_true, y_pred) if len(set(y_true)) > 1 else 0
f1 = f1_score(y_true, y_pred) if len(set(y_true)) > 1 else 0

# Stampa dei risultati
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Precision: 0.0003
Recall: 1.0000
F1-score: 0.0005
